# Numerical Benchmarks

In [2]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover Map

### Generate True Model, Full POVM and Inital

In [8]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n
    spam_target = SPAM(d=d)

    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, noise=0):
    n = spam_target.n
    inputs_spam, _ = generate_pauliInput_circuits(n, N_spam)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, inputs_spam)
    targets_spam = measurement(state, povm = spam_target.povm)

    #add noise
    targets_spam = add_noise_to_probs(targets_spam, noise=noise)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, noise=0):
    n = channel_target.n
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            trace=False,
                                            N=N_map)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis, spam_target.povm)
    
    return inputs_map, targets_map


def povm_fidelity(spam_a, spam_b):
    d = spam_a.d
    povm_a, povm_b = spam_a.povm, spam_b.povm
    ab = tf.matmul(povm_a, povm_b)
    ab_sqrt = tf.linalg.sqrtm(ab)
    fidelity = tf.math.reduce_sum(tf.linalg.trace(ab_sqrt))/d
    return fidelity


def model_pipeline(channel_target, noise):
    # Make Benchmark
    #################################################################################
    n = 3
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.8, c2=0.8)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, noise=noise)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=2000, noise=noise)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(d=d,
                      use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))

    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 300,
                        verbose = False,
                        )

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 2000,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = channel_fidelity_loss),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = 2000,
                N = 500,
                )
    #################################################################################

    return model

In [4]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = d**2
c1 = 0.9
c2 = 0.9

#prep error and full POVM error
spam_target = SPAM(d=d)

init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

spam_target = SPAM(d=d,
                  init = init_target,
                  povm = povm_target)

kraus_target = CompactLindbladMap(d, 32)

### Generate Synthetic Data with Noise

In [5]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

#add noise
targets_spam = add_noise_to_probs(targets_spam, 0.01)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None,  
                                        trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_channel(state)
targets_map = measurement(state, U_basis, spam_target.povm)

#add noise
targets_map = add_noise_to_probs(targets_map, 0.01)

### Fit Model, Initial and Corruption Matrix

In [4]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
#                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 300,
                    verbose = False,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 verbose = True,
                )

  0%|          | 0/300 [00:00<?, ?it/s]

6.335375165672933e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.021171585966863922
1 0.019265344871046856
2 0.017505008272682525
3 0.015631618948091397
4 0.013678901986562549
5 0.011736645075768696
6 0.00988431729486943
7 0.008190259744459359
8 0.006780413423634392
9 0.005743670317081307
10 0.005067930928810016
11 0.004660745466907894
12 0.004424165797797745
13 0.004249467854541899
14 0.004095797441625909
15 0.003960551935444211
16 0.0038279943292971554
17 0.0036779674390891304
18 0.0035055363314486325
19 0.0032966172925941945
20 0.0030555701908967945
21 0.0028112317778654105
22 0.002577285387474243
23 0.0023552795059432145
24 0.00215444659729896
25 0.0019902358083294844
26 0.0018614479143566525
27 0.0017631535547622283
28 0.0016901887053013917
29 0.0016396053177482458
30 0.0016009030786115024
31 0.0015664609152410732
32 0.0015316469069989024
33 0.001493122342886205
34 0.0014470456104899063
35 0.0013936548805128385
36 0.0013382842187677588
37 0.001286205032962179
38 0.0012395639220771399
39 0.0011980649642770885
40 0.001160869271360094
41 0.001

324 0.00034761544590745583
325 0.000347650890471393
326 0.00034761357257853627
327 0.0003475407942598269
328 0.00034753060527875315
329 0.00034757762574466237
330 0.00034758438392453947
331 0.0003475198271726589
332 0.00034746710984867027
333 0.0003475027839759581
334 0.00034756788982527945
335 0.00034754387536957627
336 0.0003474728915188514
337 0.00034749117434213503
338 0.00034758113803530407
339 0.00034760265328084643
340 0.000347534729605144
341 0.0003475083366689783
342 0.00034759174863924073
343 0.0003476826677404386
344 0.0003476632808677261
345 0.0003476043729150267
346 0.0003476671338542399
347 0.0003478092178241596
348 0.00034783837066144907
349 0.0003477645970943628
350 0.00034778294064405883
351 0.00034793786451117043
352 0.0003480505812805404
353 0.0003480197153033579
354 0.0003479953814174372
355 0.00034813579698952647
356 0.00034833109301321476
357 0.0003483521602230656
358 0.0003482830766171914
359 0.0003483876964462252
360 0.0003486368862418251
361 0.00034875271980634

636 0.0003343391943117929
637 0.0003339920786705864
638 0.00033447014276809624
639 0.0003340943902776655
640 0.000334602765722037
641 0.0003342105575031395
642 0.0003347629211500604
643 0.00033433970878990986
644 0.00033493335844762454
645 0.00033448310059896227
646 0.0003351425032880068
647 0.0003346438845373921
648 0.0003353712079708568
649 0.00033482422899239325
650 0.00033564956667205155
651 0.0003350268887155475
652 0.00033596037684412226
653 0.00033525595057176693
654 0.00033633683359630414
655 0.00033551356330959803
656 0.0003367649423627378
657 0.0003358072820841699
658 0.00033728594762492996
659 0.00033647687428043423
660 0.0003386145218369348
661 0.00033766186489394623
662 0.00034052015269600284
663 0.00033907761777094716
664 0.00034289007111775713
665 0.0003406869066792922
666 0.00034606749498277653
667 0.00034409006712706993
668 0.00035193563561668987
669 0.0003498522561180065
670 0.0003632482208313143
671 0.0003619458619551747
672 0.00038034393433344866
673 0.0003702210789

950 0.00032934742662522426
951 0.00032934556405150346
952 0.0003293242879307064
953 0.00032933411202986816
954 0.00032931921203560214
955 0.00032930795352195506
956 0.000329318198589846
957 0.0003292911457264635
958 0.0003292955517413342
959 0.0003292962943659533
960 0.00032926851177996576
961 0.0003292777811280567
962 0.000329265925853096
963 0.00032925511251985284
964 0.00032925666279417614
965 0.0003292351682428705
966 0.00032924105237294867
967 0.00032922990768178797
968 0.0003292153218646104
969 0.00032922539615429
970 0.0003292031095203571
971 0.0003292036391984659
972 0.00032920214618894627
973 0.00032918068710157127
974 0.00032919417775993757
975 0.0003291765947690837
976 0.0003291635853408988
977 0.000329173271800608
978 0.0003291520964299283
979 0.0003291527399177073
980 0.00032914523490582107
981 0.00032913157072029766
982 0.00032913657784676843
983 0.00032911723428970854
984 0.0003291190946219673
985 0.00032911509751706045
986 0.0003290975343854738
987 0.0003291064165933222